In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
from pandas import ExcelWriter
from tqdm import tqdm
import json
import ast
from tqdm import tqdm

def xml2Excel(inDirectory=None, outFileName=None, outDirectory=None):
    path = inDirectory
    rawText1 = open(inDirectory).read()
    rawText = open(inDirectory).read()
    soup = BeautifulSoup(rawText, 'xml')
    rawTexts = []
    tags = []
    original_text = []
    for record in soup.find_all('RECORD'):
        text = record.text
        rawTexts.append(text)
        start = 0
        tag = []
        for phi in record.find_all('PHI'):
            start = text.find(phi.text, start)
            end = start + len(phi.text)
            tagType = phi['TYPE']
            tag.append({'start': str(start), 'end': str(end), 'text': phi.text, 'TYPE': tagType})
        tags.append(tag)
        original_text.append(record)
    print(len(rawTexts), len(tags), len(original_text))
    fullPath = outDirectory + outFileName
    writer = ExcelWriter(outFileName)
    df = pd.DataFrame({'Summaries': rawTexts, 'tags' : tags, 'Annotated_Summaries' : original_text})
    df.to_excel(writer,'Sheet1', index = False)
    writer.close()

xml2Excel(inDirectory='gemini_generated_data.xml', outDirectory='.',
         outFileName='gemini_generated_summary.xlsx')

dfTrain = pd.read_excel('/content/gemini_generated_summary.xlsx')

#Llama Summaries

In [ ]:
# from bs4 import BeautifulSoup
# import pandas as pd
# from pandas import ExcelWriter

# def xml2Excel(inDirectory=None, outFileName=None, outDirectory=None):
#     path = inDirectory
#     rawText1 = open(inDirectory).read()
#     rawText = open(inDirectory).read()
#     soup = BeautifulSoup(rawText, 'xml')
#     rawTexts = []
#     tags = []
#     original_text = []
#     for record in soup.find_all('RECORD'):
#         text = record.text
#         rawTexts.append(text)
#         start = 0
#         tag = []
#         for phi in record.find_all('PHI'):
#             start = text.find(phi.text, start)
#             end = start + len(phi.text)
#             tagType = phi.get('TYPE')  # Use .get() to retrieve attribute value or None if it doesn't exist
#             if tagType:  # Check if 'TYPE' attribute exists
#                 tag.append({'start': str(start), 'end': str(end), 'text': phi.text, 'TYPE': tagType})
#             else:
#                 # Handle case where 'TYPE' attribute is missing
#                 tag.append({'start': str(start), 'end': str(end), 'text': phi.text, 'TYPE': None})
#         tags.append(tag)
#         original_text.append(record)
#     print(len(rawTexts), len(tags), len(original_text))
#     fullPath = outDirectory + outFileName
#     writer = ExcelWriter(outFileName)
#     df = pd.DataFrame({'Summaries': rawTexts, 'tags' : tags, 'Annotated_Summaries' : original_text})
#     df.to_excel(writer,'Sheet1', index = False)
#     writer.close()

# xml2Excel(inDirectory='/content/1015_gemini_final.xml', outDirectory='.',
#          outFileName='1596_summary_gemini1.xlsx')

1045 1045 1045


In [ ]:
dfTrain = pd.read_excel('/content/1596_summary_gemini_final.xlsx')

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
dfTrain

In [ ]:
def func(txt, tgt):
  words = []
  ctag  = []
  j = 0
  for i in tgt:
    k = int(i['start'])
    l = int(i['end'])
    for tt in txt[j:k].strip().split(" "):
      tt = tt.replace("\n", '')
      tt = tt.replace("\n", '').strip()
      if tt != '':
        words.append(tt)
        ctag.append("O")
    info = i["text"].split(" ")
    ctag.append(f"B-{i['TYPE']}")
    for _ in info[1:]:
      ctag.append(f"I-{i['TYPE']}")
    words += [ii.strip() for ii in info]
    j = l
  for tt in txt[j:len(txt)].strip().split(" "):
    tt = tt.replace("\n", '')
    tt = tt.replace("\n", '').strip()
    if tt != '':
      words.append(tt)
      ctag.append("O")
  return words, ctag

In [ ]:
dfTrain['word_count'].describe()

count    1596.000000
mean      373.801378
std       214.105871
min       100.000000
25%       260.750000
50%       349.500000
75%       449.250000
max      4256.000000
Name: word_count, dtype: float64

In [ ]:
df_train = {'Doc #' : [], 'Words' : [], 'Tags' : [],'segment #' : []}

for i in tqdm(range(dfTrain.shape[0])):
  txt = dfTrain .iloc[i]['Summaries']
  tgt = ast.literal_eval(dfTrain .iloc[i]['tags'])
  a,b = func(txt, tgt)
  df_train["Words"] += a
  df_train["Tags"] += b
  df_train['Doc #'] += [f"Doc-{i}"]* len(a)
  x = int(len(a)/100)
  y = len(a)%100
  for j in range(x):
    df_train['segment #'] += [f"segment {i} {j}"]*100
  df_train['segment #'] += [f"segment {i} {j+1}"]*y
df_train = pd.DataFrame(df_train)

100%|██████████| 1596/1596 [00:01<00:00, 1332.08it/s]


In [ ]:
dfTrain.to_excel('1015_llama_summaries.xlsx', index=False)

In [ ]:
df_train

,Doc #,Words,Tags,segment #
0,Doc-0,\n,O,segment 0 0
1,Doc-0,Max,B-Hospital_Name,segment 0 0
2,Doc-0,Super,I-Hospital_Name,segment 0 0
3,Doc-0,Specialty,I-Hospital_Name,segment 0 0
4,Doc-0,Hospital,I-Hospital_Name,segment 0 0
...,...,...,...,...
259015,Doc-618,to,O,segment 618 2
259016,Doc-618,follow,O,segment 618 2
259017,Doc-618,a,O,segment 618 2
259018,Doc-618,low-sodium,O,segment 618 2


In [ ]:
df_train.to_csv("gemini.csv")

In [ ]:
x= pd.read_csv("/content/gemini.csv")

In [ ]:
x

,Unnamed: 0,Doc #,Words,Tags,segment #
0,0,Doc-0,\n\n,O,segment 0 0
1,1,Doc-0,244239321,B-ID,segment 0 0
2,2,Doc-0,\n,O,segment 0 0
3,3,Doc-0,KGH,B-HOSPITAL,segment 0 0
4,4,Doc-0,\n,O,segment 0 0
...,...,...,...,...,...
634253,634253,Doc-1595,Dr.,O,segment 1595 6
634254,634254,Doc-1595,Wall,B-DOCTOR,segment 1595 6
634255,634255,Doc-1595,\n[,O,segment 1595 6
634256,634256,Doc-1595,report_end,O,segment 1595 6


In [ ]:
x['Words'] = x['Words'].replace('\\n\\n', '')
x['Words'] = x['Words'].replace('\\n\\n\\n', '')
x['Words'] = x['Words'].replace('\\n\\n\\n\\n', '')
x['Words'] = x['Words'].replace('\\n[', '')
x['Words'] = x['Words'].replace('\\n', '')
x['Words'] = x['Words'].replace('\\n[', '')
x['Words'] = x['Words'].str.replace(r'^[\/;,.:\[\]*\\n**]+|[\/;,.:\[\]*\\n**]+$', '')
x['Words'] = x['Words'].str.replace(r'^[\/;,.:\[\]*]+|[\/;,.:\[\]*]+$', '')

In [ ]:
x.isnull().sum()

Unnamed: 0     0
Doc #          0
Words         61
Tags           0
segment #      0
dtype: int64

In [ ]:
x= x.dropna()

In [ ]:
x['Tags'].value_counts()

Tags
O                599245
B-ID               7504
B-DATE             6242
I-DOCTOR           5269
B-DOCTOR           3522
B-HOSPITAL         2304
I-PATIENT          2233
B-PATIENT          2043
I-HOSPITAL         1247
I-LOCATION          798
B-LOCATION          516
I-ADDRESS           491
B-CONTACT           471
B-PHONE             437
B-AGE               414
I-DATE              319
I-ID                245
I-PHONE             184
I-CONTACT           146
B-ADDRESS           107
B-EMAIL              95
I-AGE                85
I-NAME               69
B-NAME               62
I-RX                 30
B-GENDER             17
B-DISEASE            14
B-SEX                12
I-DISEASE             9
I-ADDRESS1            7
I-ADDRESS2            7
I-TEST                6
B-DRUG                5
B-SPECIALITY          4
B-DOB                 3
B-TIME                3
I-TIME                3
B-TEST                3
B-LOC                 3
B-RATE                2
I-SIGNATURE           2
I-SPECIALIT

In [ ]:
x.to_csv("gemini_generated_summary_original_tags.csv")

#Tag mapping to the tag set of n2c2 2006 Dataset

In [ ]:
def update_tags(df):
    df['Tags'][df['Tags'] == 'I-ADDRESS'] = 'I-LOCATION'
    df['Tags'][df['Tags'] == 'B-CONTACT'] = 'B-PHONE'
    df['Tags'][df['Tags'] == 'I-CONTACT'] = 'I-PHONE'
    df['Tags'][df['Tags'] == 'B-ADDRESS'] = 'B-LOCATION'
    df['Tags'][df['Tags'] == 'B-EMAIL'] = 'B-PHONE'
    df['Tags'][df['Tags'] == 'I-NAME'] = 'I-PATIENT'
    df['Tags'][df['Tags'] == 'B-NAME'] = 'B-PATIENT'
    df['Tags'][df['Tags'] == 'I-RX'] = 'O'
    df['Tags'][df['Tags'] == 'B-DISEASE'] = 'O'
    df['Tags'][df['Tags'] == 'B-GENDER'] = 'O'
    df['Tags'][df['Tags'] == 'I-ADDRESS1'] = 'I-LOCATION'
    df['Tags'][df['Tags'] == 'I-ADDRESS2'] = 'I-LOCATION'
    df['Tags'][df['Tags'] == 'I-TEST'] = 'O'
    df['Tags'][df['Tags'] == 'B-DRUG'] = 'O'
    df['Tags'][df['Tags'] == 'B-SPECIALITY'] = 'O'
    df['Tags'][df['Tags'] == 'B-DOB'] = 'B-DATE'
    df['Tags'][df['Tags'] == 'B-TIME'] = 'O'
    df['Tags'][df['Tags'] == 'I-TIME'] = 'O'
    df['Tags'][df['Tags'] == 'B-TEST'] = 'O'
    df['Tags'][df['Tags'] == 'B-LOC'] = 'B-LOCATION'
    df['Tags'][df['Tags'] == 'B-RATE'] = 'O'
    df['Tags'][df['Tags'] == 'I-SIGNATURE'] = 'O'
    df['Tags'][df['Tags'] == 'I-SPECIALITY'] = 'O'
    df['Tags'][df['Tags'] == 'B-DOSAGE'] = 'O'
    df['Tags'][df['Tags'] == 'I-LOC'] = 'I-LOCATION'
    df['Tags'][df['Tags'] == 'B-ADDRESS2'] = 'B-LOCATION'
    df['Tags'][df['Tags'] == 'B-ADDRESS1'] = 'B-LOCATION'
    df['Tags'][df['Tags'] == 'I-EMAIL'] = 'I-PHONE'
    df['Tags'][df['Tags'] == 'B-URL'] = 'B-PHONE'
    df['Tags'][df['Tags'] == 'B-RX'] = 'O'
    df['Tags'][df['Tags'] == 'B-SIGNATURE'] = 'O'
    df['Tags'][df['Tags'] == 'B-PERCENTAGE'] = 'O'
    df['Tags'][df['Tags'] == 'B-TEMPERATURE'] = 'O'
    df['Tags'][df['Tags'] == 'I-SEX'] = 'O'
    df['Tags'][df['Tags'] == 'B-COUNTRY'] = 'B-LOCATION'
    df['Tags'][df['Tags'] == 'B-STATE'] = 'B-LOCATION'
    df['Tags'][df['Tags'] == 'B-CITY'] = 'B-LOCATION'
    df['Tags'][df['Tags'] == 'I-DRUG'] = 'O'
    df['Tags'][df['Tags'] == 'B-SEX'] = 'O'
    df['Tags'][df['Tags'] == 'I-DISEASE'] = 'O'


    return df

In [ ]:
xy = update_tags(x)

<ipython-input-29-3fc0391f6145>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Tags'][df['Tags'] == 'I-ADDRESS'] = 'I-LOCATION'
<ipython-input-29-3fc0391f6145>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Tags'][df['Tags'] == 'B-CONTACT'] = 'B-PHONE'
<ipython-input-29-3fc0391f6145>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Tags'][df['Tags'] == 'I-CONTACT'] = 'I-PHONE'
<ipython-input-29-3fc0391f6145>:5: SettingW

In [ ]:
xy['Tags'].value_counts()

Tags
O             599365
B-ID            7504
B-DATE          6245
I-DOCTOR        5269
B-DOCTOR        3522
B-HOSPITAL      2304
I-PATIENT       2302
B-PATIENT       2105
I-LOCATION      1305
I-HOSPITAL      1247
B-PHONE         1004
B-LOCATION       631
B-AGE            414
I-PHONE          331
I-DATE           319
I-ID             245
I-AGE             85
Name: count, dtype: int64

In [ ]:
xy.to_csv("gemini_mapped_to_n2c2_2006.csv")